In [36]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path as op
import shutil
from zipfile import ZipFile
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from utils import get_image_paths, word_from_image_path, preprocess_image, print_im, TextTransform, N_CHARS, SEQUENCE_LENGTH, IMAGE_DIMENSIONS

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
images_paths = get_image_paths()

In [57]:
images = get_image_paths()

In [39]:
text_transformer = TextTransform()

### creating a batch

In [58]:
batch_y = text_transformer.make_batch_labels(images)

In [60]:
ims = [load_img(im, grayscale=True) for im in images]
ims = [preprocess_image(img_to_array(im)) for im in ims]
batch = np.array(ims)

### Model creation

In [12]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Reshape, Activation
from keras.layers import Flatten

In [68]:
convolutions = [64, 128, 256, 512, 512]
kernels = [5, 5, 3, 3, 3]

model = Sequential()
input_shape = (None,) + IMAGE_DIMENSIONS + (1,)

model.add(Conv2D(nb_filter=64,
                     nb_row=kernels[0],
                     nb_col=kernels[0],
                     activation='relu',
                     border_mode='same',
                     batch_input_shape=input_shape, name="convo" + str(0)))

model.add(MaxPooling2D(pool_size=(2, 2), border_mode='same'))

for i, (kernel, convolution_size) in enumerate(zip(convolutions[1:], kernels[1:])):
    model.add(Conv2D(nb_filter=convolution_size,
                     nb_row=kernel,
                     nb_col=kernel,
                     activation='relu',
                     border_mode='same',
                     name="convo" + str(i + 1)))
    
    if i <= 3 :
        model.add(MaxPooling2D(pool_size=(2, 2), border_mode='same',))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
# model.add(Dense(4096, activation='relu'))
# model.add(Dense(4096, activation='relu'))

model.add(Dense(SEQUENCE_LENGTH * N_CHARS))

model.add(Reshape((SEQUENCE_LENGTH, N_CHARS)))
model.add(Activation('softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy')

In [69]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convo0 (Convolution2D)           (None, 10, 30, 64)    1664        convolution2d_input_9[0][0]      
____________________________________________________________________________________________________
maxpooling2d_37 (MaxPooling2D)   (None, 5, 15, 64)     0           convo0[0][0]                     
____________________________________________________________________________________________________
convo1 (Convolution2D)           (None, 5, 15, 5)      5242885     maxpooling2d_37[0][0]            
____________________________________________________________________________________________________
maxpooling2d_38 (MaxPooling2D)   (None, 3, 8, 5)       0           convo1[0][0]                     
___________________________________________________________________________________________

In [70]:
batch.shape

(501, 10, 30)

In [71]:
%%time
res = model.predict(batch.reshape((batch.shape[0],) + IMAGE_DIMENSIONS+ (1,)))

CPU times: user 2min 31s, sys: 1.74 s, total: 2min 32s
Wall time: 48.6 s


In [72]:
res.shape

(501, 23, 37)

In [52]:
y = np.random.randn(4096 * 10).reshape((10, 4096))

In [ ]:
model.fit(batch.reshape((batch.shape[0],) + IMAGE_DIMENSIONS + (1,)), batch_y, nb_epoch=100)

Epoch 1/100
501/501 [==============================] - 282s - loss: 3.4862    
Epoch 2/100
501/501 [==============================] - 308s - loss: 3.4355    
Epoch 3/100
501/501 [==============================] - 291s - loss: 3.3457    
Epoch 4/100
501/501 [==============================] - 285s - loss: 3.2078    
Epoch 5/100
501/501 [==============================] - 288s - loss: 3.0186    
Epoch 6/100
384/501 [=====================>........] - ETA: 67s - loss: 2.8068